<a href="https://colab.research.google.com/github/satvik314/langchain/blob/main/ConversationRetreivalChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install langchain unstructured[local-inference] openai chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 KB 

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, OnlinePDFLoader


In [2]:
loader = UnstructuredPDFLoader("/content/cognitive_psy.pdf")
documents = loader.load()

In [4]:
import os
from getpass import getpass
os.environ['OPENAI_API_KEY'] = getpass()

··········


In [10]:
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

In [14]:
retreiver = vectorstore.as_retriever()

In [16]:
qa = ConversationalRetrievalChain.from_llm(llm = OpenAI(temperature = 0), retriever = retreiver)

In [21]:
chat_history = []
query = "What is the document about?"

result = qa({"question" : query, "chat_history" : chat_history})

In [22]:
result["answer"]

" The document is about the scientific study of human memory and cognition, including topics such as perception, decision making, recognizing letters of the alphabet, and having conversations. It also discusses the history of the study of the mind, from ancient works such as Homer's Iliad and Odyssey to the cognitive revolution of the 1950s."

In [23]:
chat_history = [(query, result["answer"])]
query = "Can you elaborate more?"
result = qa({"question": query, "chat_history" : chat_history})
result["answer"]

' This document is about human memory and cognition, and specifically about the scientific study of human memory and cognition. It covers topics from basic perception through complex decision making, from seemingly simple mental acts such as recognizing a letter of the alphabet to very complicated acts such as having a conversation. It also mentions a book by Miller, Galanter, and Pribram called Plans and the Structure of Behavior, which suggests that human problem solving can be understood as a kind of planning in which mental strategies or plans guide behavior toward its goal.'

In [28]:
from IPython.display import Markdown, display

def askquestion():
  chat_history = []
  while True:
    query = input("Ask your question: ")
    result = qa({"question": query, "chat_history" : chat_history})
    chat_history = [(query, result["answer"])]
    print(result['answer'])


In [29]:
askquestion()

Ask your question: What is the document about?
 The document is about the scientific study of human memory and cognition, including topics such as perception, decision making, recognizing letters of the alphabet, and having conversations. It also discusses the history of the study of the mind, from ancient works such as Homer's Iliad and Odyssey to modern cognitive science.
Ask your question: Can you elaborate more?
 This document is about human memory and cognition, and specifically about the scientific study of human memory and cognition. It covers topics from basic perception through complex decision making, from seemingly simple mental acts such as recognizing a letter of the alphabet to very complicated acts such as having a conversation. It also mentions a book by Miller, Galanter, and Pribram called Plans and the Structure of Behavior, which suggests that human problem solving can be understood as a kind of planning in which mental strategies or plans guide behavior toward its g

KeyboardInterrupt: ignored